# Class 3: An Introduction into Functions and other Building Blocks

Created by Kevin O'Neill, edited by Abby Hsiung, and recreated by Raphael Geddert

Welcome to week 3 of Introducing Python Programming! Today, we will be learning about `functions`, what they are, how we can create them, and what they can help us do! 

Remember, in Jupyter Notebook, you hit ctrl+enter to run a code block, or shift+enter to run it and immediately select the next block.

### Objective:

To learn about how we create `functions` and how we can use both `functions` and `methods`. 

### Learning Outcomes:
- Know the building blocks for defining `functions`
- Learn about how to use `functions` to transform data
- Learn the different ways to create `functions`
- Learn the difference between `functions` and `methods`

### Homeworks:
- Finish reviewing this Notebook
- Write a project proposal with your partner by next week

---

This is also the first time we are using a "helper function(s)". Helper functions are blocks of code that you don't need to worry about understanding, they just help make some of our work in this notebook easier! 

*For a bonus though, you can practice annotating (using comments `#`) to try and understand what is happening in the function below*

In [ ]:
#@title Helper functions (click play before starting the notebook!)
import math, random, time

def printStimuli(*args, buffer=' ', width=30, clearLines=1):
  """Print one or more strings centered within a buffer of size 'width', 
  deleting 'clearLines' lines of text above the printed text.

  Note: this does *not* print a newline after the output.
  """
  if sum(list(map(len, args))) > width:
    raise ValueError('arguments too wide to fit in buffer')

  if len(args) == 0:
    text = buffer * width
  else:
    text = ''
    for a in args:
      text += '{:^{width}}'
    text = text.format(*args, width=math.floor(width / len(args)))

  if clearLines <= 0:
    text = '\n' + text
  else:
    text = '\b' * (width*clearLines) + text

  print(text, end='')

## Review of Lists and Loops

In the last lab, we talked about how to run a block of code multiple times using loops and lists. Loops are great because you can turn a long list of repetitive code like this:

In [ ]:
file1 = 'image_01.png'
file2 = 'image_02.png'
file3 = 'image_03.png'
file4 = 'image_04.png'
file5 = 'image_05.png'

print(file1, file2, file3, file4, file5)

into something more compact + readable like this:

In [ ]:
files = []
for i in range(5):
  files.append(f'image_0{i+1}.png')
print(files)

Now instead of five different variables to keep track of, we instead have a nice, compact single variable that is one list of filenames. But what happens if the code we need to repeat isn't so simple? What if we don't know what code we need to repeat yet? Or what if we might need to change what happens in the code later on? As we'll see, loops don't always solve our problems.

## Building Experimental Blocks


When programming an experiment, it is natural to think of things in terms of the steps that take place in each trial. A typical task looks something like this:

```python
trials = 20
for trial in range(trials):
  # 
  # display a blank screen
  # ...
  #
  # display a fixation cross
  # ...
  #
  # display some stimulus
  # ...
  #
  # record a response
  # ...
  #
  # give feedback
  # ...
```

That is, each trial starts with a blank screen, followed by a fixation cross. After the fixation cross a stimulus is presented and the participant makes a response. Finally, the participant is given feedback about whether their response is correct.

But with all of the code in a single for loop, things can get pretty messy. For example, here is code for a simple **delayed match to response task**. In this task, your only goal is to remember a stimulus, then report it after a short delay. The code uses a helper function `printStimuli` to display text to the screen. Don't worry about figuring out all the details for now, but see if you can make sense of what the code is doing:

(We're going to first go through the code at a general, overview level. We will then step closer to see what each piece of the code is doing!)

In [ ]:
## experiment parameters
letters = ['A', 'B', 'C', 'D']    # stimuli that can appear
trials = 3                        # number of trials
display_time = 0.2                # time (s) that the stimulus is shown
delay = 3                         # time (s) between stimulus presentation & recognition
ITI = 1                           # inter-trial interval (s)

## experiment data (start off empty)
data = {'target': [None]*trials,
        'lure': [None]*trials,
        'correct': [None]*trials}

for trial in range(trials):
  # randomly choose two stimuli (the first will be the target)
  stim = random.sample(letters, 2)
  data['target'][trial] = stim[0]
  data['lure'][trial] = stim[1]

  # display a blank screen
  printStimuli('')
  time.sleep(ITI)

  # display a fixation cross
  printStimuli('+')
  time.sleep(ITI)

  # display the target stimulus
  printStimuli(data['target'][trial])
  time.sleep(display_time)

  # display a blank screen during the delay
  printStimuli()
  time.sleep(delay)

  # display the two choices in random order
  if random.random() > .5:
    printStimuli(data['target'][trial], data['lure'][trial])
  else:
    printStimuli(data['lure'][trial], data['target'][trial])
  
  # get the participant's response
  text = input()
  print('\b', end='')
  
  if text.upper() == stim[0]:
    printStimuli('Correct :)')
    data['correct'][trial] = 1
  else:
    printStimuli('Incorrect :(')
    data['correct'][trial] = 0
  time.sleep(ITI)

printStimuli(f"You got {sum(data['correct'])} out of {trials} correct.")


It's not that easy, is it? When there's that much code inside of our loop, that makes it hard to figure out where variables are initialized and what each line of code does. While the code *works*, it's not very intuitive. Even if you wrote the code yourself, you'd probably have a hard time going through it in a year or two. The question is, is there a better way?

# Functions

As we saw above, sometimes it can get clunky to have all of your code laid out inside a for loop if a lot happens inside of that loop. To figure out how to solve this problem, we need functions. Before using functions to make our lives easier, however, we have to first understand **what functions are.**

## What is a function?
Simply put, a function maps inputs to outputs. So basically, you _give_ a function something, the function does its function-y things and _spits out_ something transformed. 

You've already encountered lots of functions already, actually! Take the `max` function, for instance, which takes some numbers and tells you which number is the largest:

In [ ]:
print(max)
print(type(max))

As you can see, these are called "build-in functions" because they come standard with base Python.

Here, we are just printing the `max` function itself. Python tells us that this is a function, but since there are no parentheses after the `max`, we have not called (or executed) the function yet.

In [ ]:
max(1, 2, 3)

In this block, we *called* the `max` function by giving it some *arguments* (1, 2, and 3) in parentheses. When we call a function on some arguments, we get a *return value*, which in this case corresponds to the largest argument given to `max` (3).

In the case of the `max` function, it's pretty obvious from the name of the function what the function does: it gives the max of some numbers. But when it isn't very obvious, we can always ask Python for some help:

In [ ]:
help(max)

This documentation says that there are two definitions of `max`: one that uses an iterable (like a list), and one that takes the numbers individually. This lets us know that if we want to, we can also call `max` this way:

In [ ]:
max([1, 2, 3])

## Who needs functions?

When programming, we use functions for three main reasons: **code duplication**, **abstraction**, and **efficiency**. Let's take a look at each of these reasons individually.

### **Code duplication**

One huge reason that functions are so important is that when we write a function, we know that we will never have to write that same piece of code again. Loops can help us limit the amount of code that we duplicate, but they can only go so far.

Consider the `max` function again: what would it be like to program without this function? What would you need to do? For instance, let's say we have some lists storing participants' average task accuracy, their average response time, their age, and their IQ, and we want to compute the maximum of each variable. Without the `max` function, we would need to figure out how to calculate the maximum of a list and write out the code to do that every single time we wanted to calculate a maximum.

Here is the code to find the max of a list of numbers manually.

In [ ]:
my_list_of_numbers = [1,2,4,3,5,2]

max_num = my_list_of_numbers[0]
print(f'Max num starts as {max_num}')
for num in my_list_of_numbers:
    print(f'Trying num: {num}')
    if num > max_num:
        print(f'{num} is greater than max_num ({max_num}). Setting max_num to {num}.')
        max_num = num

What is we had lots of variables in an experiment? We might need to find the max of all of these numbers. Here is what that would look like:

In [ ]:
accuracy = [.54, .83, .39, .60, .92]
rt = [2.4, 1.2, 5.6, 4.1, 3.9]
age = [19, 22, 27, 32, 20]
iq = [122, 118, 117, 132, 146]

max_accuracy = accuracy[0]
for a in accuracy:
  if a > max_accuracy:
    max_accuracy = a

max_rt = rt[0]
for r in rt:
  if r > max_rt:
    max_rt = r

max_age = age[0]
for ag in age:
  if a > max_age:
    max_age = a

max_iq = iq[0]
for i in iq:
  if i > max_iq:
    max_rt = i

print(max_accuracy) #max should be .92
print(max_rt) #max should 5.6
print(max_age) #max should be 32
print(max_iq) #max should be 146

There are two major problems with this. First, it makes your code much, much longer! If we used the `max` function, we would only need a single line to calculate each maximum. Second, doing things this way is *effortful*, and almost always leads to bugs in your code. For example, **the code above actually has two different bugs! Can you find them?** These bugs are hard to find because the duplicated code looks so similar, and because if you need to make changes to the duplicated code, it's very hard to remember every single place you need to make this change to.

Instead, we of course just use the max function. This makes the code much much shorter, easier to read, and best of all, much easier to find bugs in.

In [ ]:
accuracy = [.54, .83, .39, .60, .92]
rt = [2.4, 1.2, 5.6, 4.1, 3.9]
age = [19, 22, 27, 32, 20]
iq = [122, 118, 117, 132, 146]

print(max(accuracy))
print(max(rt))
print(max(age))
print(max(iq))

### **Abstraction**

The other huge reason that functions are so great is that after we write a function, we no longer need to think about how the code inside the function works: we can just focus on what matters *at the current level of abstraction*.

To get a good idea of what this means, let's return to the idea of building experimental blocks. As a reminder, most tasks can be construed as a loop over trials, which looks something like this:

```python
trials = 20
for trial in range(trials):
  # display a blank screen
  # ...
  #
  # display a fixation cross
  # ...
  #
  # display some stimulus
  # ...
  #
  # record a response
  # ...
  #
  # give feedback
  # ...
```

If we try to program this task without functions, we will need all of the code to display each stimulus directly inside the loop. As we will see later on, displaying each stimulus can take a few lines of code, involving lots of  variables. Not only is this a lot of duplicate code, but it leads to another problem: if we have so much code inside our `for` loop, it is difficult to figure out what the loop is for without reading every single line of code, considering how each stimulus is drawn on the screen, how responses are recorded, etc. It is much better to have your task programmed something like this:

```python
trials = 20
for trial in range(trials):
  showStimulus('', 1)          # display a blank screen for 1s
  showStimulus('+', 1)         # display a fixation cross for 1s
  showStimulus('', 2)          # display some stimulus for 2s
  
  response = recordResponse()  # record a response
  
  # give feedback for 1s
  if isCorrect(response):
    showStimulus('correct', 1)
  else:
    showStimulus('incorrect', 1)
```

In addition to being much shorter, this is easier to read because it **abstracts** the unnecessary details away from loop. That is, when we read this loop, we don't need (or want) to know *how* stimuli are drawn on the screen, we just want to know which stimuli are drawn on the screen, in what order, etc. Likewise, we don't want to have to think about *how* the responses are recorded, it is enough to just know that they are being recorded.

### **Efficiency**

Finally, functions are great because Python already has functions for so many things you need to do, and these existing functions will almost always be faster and more memory efficient than whatever code you could work out in a day or two. Efficiency is a source of pride for software developers, and so they spend a *lot* (really, a lot) of time making their code as efficient as possible. So why bother trying to write code yourself when better, faster code is already out there?

To demonstrate this, let's assume that we want the average of a giant list of numbers. Here we will use the `random` module to generate a giant list, and use the `time` module to compare the runtime of two different approaches: using a `for` loop to calculate the average manually and using the `sum` function.

In [ ]:
l = [1]*10000000

print(f'{"Algorithm" : >10}{"Result" : ^10}{"Time" : ^10}') # <, ^: formatting cues

### VERSION 1: FOR LOOP ###
# for loop version for calculating the average
start = time.time()
list_sum = 0
# loop through each item and add to total to find the sum
for item in l:
  list_sum = list_sum + item
avg = list_sum / len(l)
stop = time.time()
print(f'{"for loop" : >10}{avg : ^10.2f}{stop - start : ^10.4f}') # 10.2, 10.4: number of decimal places

### VERSION 2: SUM FUNCTION ###
start = time.time()
avg = sum(l) / len(l)
stop = time.time()
print(f'{"sum" : >10}{avg : ^10.2f}{stop - start : ^10.4f}')

As we can see, the sum function isn't just less code, but it actually runs over 10 times faster than the loop version!

## How do I make a function?

Hopefully by now you are convinced that functions are important programming tools that can make your life easier. There are two ways to make a function in Python. 

We can:
- Make a function with `def`
- Make a function with `lambda`

These ways are not synonymous, though and have important differences. Let's talk about each individually.


### Making functions with `def`

In most cases, you'll want to use the `def` keyword to write a new function. Functions defined with `def` look something like this:

```python
def my_function(argument1, argument2, ..., ):
  """ A documentation string that tells you what 
  the function does and how to use it"""
  # ...some code...
  return value
```
Importantly, the way you specify the code in `(argument1, argument2, ...)` tells you how to reference those variables in the `...some code...` part.


As an example, let's say we want to make a function for the average of a list of numbers. We would probably write a function like this:

In [ ]:
def average(l):
  """Calculate the average of some numbers"""
  return sum(l) / len(l) # l here (and up where it's defined) is like the placeholder for an eventual datatype

Now that we have this function, it's easy to calculate the average of a bunch of lists!

We were interested earlier in finding the max from these lists, but using our new function, we can also find the average!

In [ ]:
accuracy = [.54, .83, .39, .60, .92]
rt = [2.4, 1.2, 5.6, 4.1, 3.9]
age = [19, 22, 27, 32, 20]
iq = [122, 118, 117, 132, 146]

print(average(accuracy))
print(average(rt))
print(average(age))
print(average(iq))

Since we included the **documentation string** in our function definition (that is what the """Calculate the ...""" thing is), we can also call `help` if we forget how to use it:

In [ ]:
help(average)

If you're making complex functions, it's often helpful in the documentation string to include the input variables your function will take as well as the expected output. You won't do this so much this semester, but you can imagine how helpful this would be if somebody else was ever going to borrow your code!

### **Exercise**

Now it is your turn! For this exercise, we are going to try to create our own mini experiment using functions.

The experiment is this: a participant (you) will see a digit between 1 and 9 (not including 5). Then, they will need to indiate if it is greater or less than 5. If they get it correct, we will say "yay!". If they get it incorrect, we will say "Boo!".

Below are the parts of your experiment. It is up to you to create the functions necessary to make this work!

```python
stimulus = choose_stimulus() #pick a number between 1 and 9, not including 5

display_stimulus(stimulus) #print the number

response = get_response() #use input to get a response

show_feedback(stimulus, response) #show if that response was correct
```

First, let's write a function called `choose_stimulus`, that returns a stimulus between 1 and 9 (not including 5). We will randomly choose a digit using `random.choice(list)`, where list is the list of numbers you'd like to choose from.

---



In [ ]:
#create a function choose_stimulus
def choose_stimulus():
  num_list = ...
  stimulus = ...
  return 

Test the function below!

In [ ]:
choose_stimulus()

Great, lets display this stimulus! Create a function called `display_stimulus` that prints the stimulus. Note that we are using an input here, which is the number we would like to print.

In [ ]:
#create a function display_stimulus

Test it!

Great! Now let's create a function called `get_response` to ask for a response from a participant. We can do this using `input(instruction text here)`. Ask for either a "y" or a "n".

In [ ]:
#create a function get_response

And test it (always!)

Great job! Now we are really getting somewhere. Finally we need a function called `show_feedback` to show feedback depending on if the answer is correct given the stimulus or not. This is the trickiest bit. We'll need to use if statements from last week to display "Correct" or "Incorrect" based on if the number is less than 5 or not. This requires checking not only whether they said "y" or "n", but also if the number is in fact greater or less than 5.

Hint: we'll need two inputs to our function! What are they?

In [ ]:
#create a function show_feedback

You guessed it, test it!

We are finally ready to put this all together. Run the function below to see your experiment realized.

In [ ]:
stimulus = choose_stimulus()

display_stimulus(stimulus)

response = get_response()

show_feedback(stimulus, response)

**BONUS** If you have time, how would you make the above experiment run for say 5 trials using a foor loop?

**CHALLENGE BONUS** How would you rewrite the get_response() function to only accept certain answers?

# The code below is bonus, and will only be covered if time allows. Feel free to read through at your own leisure outside of class if you want more details.

#### **Variable-length arguments**

What if we want our average function to take numbers themselves, and not just a list of numbers? One way would be to write our function like this:

In [ ]:
def average(x1, x2):
  """Calculate the average of some numbers"""
  return (x1+x2) / 2

This works great anytime we want to average just two numbers. But what if we want the average of three, four, or more numbers? Or if we don't know how many numbers we will be averaging in advance? This is where **variable-length arguments** come in hand. We make a variable-length argument using the asterick `*`:

In [ ]:
def average(*args):
  """Calculate the average of some numbers"""
  return sum(args) / len(args)

Now we can pass as many numbers as we want to our average function!

In [ ]:
print(average(1))
print(average(1, 2))
print(average(1, 2, 3))
print(average(1, 2, 3, 4))

Sadly, our average function no longer works with lists anymore:

In [ ]:
average(accuracy)

Why doesn't this work? Well, using variable-length arguments works by wrapping the arguments passed to the function into a list. So if we call `average(1, 2, 3)`, then the argument `args` will be set to the list `[1, 2, 3]` which is used to calculate the average. So when we pass in a list like `average([1, 2, 3])`, the argument `args` is being set to a list with this list inside of it (`[[1, 2, 3]]`)! 

To make our `average` function work both ways, then, we can remember all the amazing things we learned during control flow and use an `if` statement to check if args is a list containing a list:

In [ ]:
def average(*args):
  """Calculate the average of some numbers"""
  if len(args) == 1 and isinstance(args[0], list):
    args = args[0]
  return sum(args) / len(args)

In [ ]:
print(len([1,2,3]))
print(len([[1,2,3]]))

Now it should work both ways!

In [ ]:
print(average(1, 2, 3))
print(average([1, 2, 3]))

#### **Keyword arguments**

Another useful feature of `def` is that you can supply default values to arguments, known as **keyword arguments**. Keyword arguments are just regular old arguments with the `=` and some default value following the name of the argument. 

For example, let's say we want a function to normalize a list of values to some range, defined by a lower and upper value. We often do this is our data analysis to help standardize responses across people.We could write our function like so:

In [ ]:
def normalize(l, lower, upper):
  """Rescale l to the range [lower, upper]"""
  scale = (upper - lower) / (max(l) - min(l))
  return [scale*(x - min(l)) + lower for x in l]

This function *works*, but it requires you to specify `min` and `max` every single time you call it:

In [ ]:
print(normalize(accuracy, 0, 1))
print(normalize(rt, 0, 1))
print(normalize(age, 0, 100))
print(normalize(iq, -1, 1))

If we try to call `normalize` without a specified range, things will break:

In [ ]:
normalize(accuracy)

Usually, though, we will want to put our values into the range `[0, 1]`. To make these values default, we can put them into our function `def`:

In [ ]:
def normalize(l, lower=0, upper=1):
  """Rescale l to the range [lower, upper]"""
  scale = (upper - lower) / (max(l) - min(l))
  return [scale*(x - min(l)) + lower for x in l]

Now if we don't specify which range to normalize our values to, the function will assume that we want it in `[0, 1]`!

In [ ]:
print(normalize(age))

Finally, it's also helpful to know that we can use the same `=` syntax when *calling* the function with keyword arguments. This helps ensure that you are passing the right value to the right arguments, even if you put them out of order:

In [ ]:
print(normalize(age, lower=-1, upper=1))
print(normalize(age, upper=1, lower=-1))

### Making anonymous functions with `lambda`

In almost every case, we will want to use `def` to write our functions, since `def` gives us access to a bunch of features like documentation strings, variable length & keyword arguments, etc. 

Sometimes, though, we will need to make a function that will only be used one time. For example, let's say that our `age` data is off by one, and so we need to add one to every number in `age`. We could decide to write out the function as such:

In [ ]:
def add_one(x):
  """Take a number x, and return x+1"""
  return x + 1

print(age)
print([add_one(x) for x in age])

That's great and all, but how often will we *really* need this `add_one` function? Probably never again. If we will never use it again, it might not seem worth the effort to use all of the nice features of `def`: it should be obvious that the code adds one to a number, so extensive documentation seems unnecessary. That's where `lambda` comes in! `lambda` defines **anonymous functions**, which are one-off functions that don't have a name (hence anonymous). Here is the same code above, using `lambda` instead of `def`:

In [ ]:
age = [19, 22, 27, 32, 20]

add_one = lambda x: x + 1

print([add_one(x) for x in age])

In this case `lambda` didn't save us a whole lot of space or time, because we still stored the function into the variable `add_one`, and we still needed a `for` loop to apply the function to every value in `age`. Thankfully, `lambda` can easily be used with functions like `map` and `filter`, which do the iteration for you!

In [ ]:
list(map(lambda x: x + 1, age))

Using a similar syntax, we can filter out the participants that are, say, over 25:

In [ ]:
list(filter(lambda x: x >= 25, age))

**BONUS:** This type of programming is known as *functional programming*, since the focus is on applying functions to variables, not on the loops that make this all work. We won't talk much more about functional programming, but you should know that in many cases, using `map` can be much faster than writing out a `for` loop yourself (though in this case the list comprehension is fastest):

In [ ]:
l = [1]*10000000

print(f'{"Algorithm" : >20}{"Time" : ^10}')

start = time.time()
l_plus_one = [None]*len(l)
for i in range(len(l)):
  l_plus_one[i] = l[i] + 1
stop = time.time()
print(f'{"for loop" : >20}{stop - start : ^10.4f}')

start = time.time()
l_plus_one = [x+1 for x in l]
stop = time.time()
print(f'{"list comprehension" : >20}{stop - start : ^10.4f}')

start = time.time()
l_plus_one = list(map(lambda x: x+1, l))
stop = time.time()
print(f'{"map" : >20}{stop - start : ^10.4f}')

## How do I make sure my function works?

One downside to writing functions is that you have to ensure that:
- the function works as intended when it is called correctly, and 
- that the function does get called correctly. 

We'll spend some time on this later in the program, but it'll help to have some of these things in the back of your mind.

### Searching for bugs

The most important thing when writing a function is to ensure that it does what you think it does when it is called properly. For instance, take our orignal average function:

By looking at the code, we can see that `average` takes a list of numbers, and it returns the sum of those numbers divided by the number of them (a.k.a., the average):

In [ ]:
def average(l):
  """Calculate the average of some numbers"""
  return sum(l) / len(l)

average([1, 2, 3])

But we decided that we wanted our `average` function to work with individual numbers, too. But our original version gave us an error here:

In [ ]:
average(1, 2, 3)

To the extent that we want `average` to work in this case, the presence of this error is a bug! To fix it, we used variable-length arguments:

In [ ]:
def average(*args):
  """Calculate the average of some numbers"""
  if len(args) == 1 and isinstance(args[0], list):
    args = args[0]
  return sum(args) / len(args)

print(average([1, 2, 3]))
print(average(1, 2, 3))

There are many kinds of different bugs that can occur in your functions, and this is just one type. But the only way to find bugs is to test your function on all of the different kinds of inputs you think it should be able to handle!

In [ ]:
print(average(42,8769,21))
print(average(1.243512345, 5, 8.765))
print(average('a', 'b', 45))

### Dealing with function misuse

Another important thing to consider when writing a function is how people might try to use your function incorrectly. For example, it doesn't make sense to calculate the average of an empty list, since there are no numbers to average. But what happens when we call our `average` function on an empty list anyway?

In [ ]:
l = []
average(l)

Python tells us that we're trying to divide by zero, and it reports an error. But if you aren't the one that wrote the average function, this error might be confusing for you: why am I getting a divide-by-zero error when I just want to calculate the average of my list? To make this easier on the user of the function, we can throw a more interpretable error:

In [ ]:
def average(*args):
  """Calculate the average of some numbers"""
  if len(args) == 1 and isinstance(args[0], list):
    args = args[0]
  
  if len(args) == 0:
    raise ValueError('cannot compute the average of 0 numbers!')
  return sum(args) / len(args)

average(l)

Now we still get an error when trying to average zero numbers, but our error message is more helpful: we know that we called the function improperly, and our list is empty when it shouldn't be.

Trying to predict all of the ways in which a function might get misused can be difficult and can make your function unnecessarily complicated. 

However, it is important to try to "break" your functions in this way so that, inevitably, when someone else breaks your function in the same way, they will immediately know what they did wrong. Later on in the semester, y'all will help each other with this process as we test and debug our difference designs. 

# Methods

All of the functions we've discussed so far are "argument-neutral" in that the functions don't *belong* to a particular argument. For instance, the `average` function gets applied over all of its arguments, and it doesn't make a whole lot of sense to say that `average` belongs to any particular argument. However, certain functions, known as **methods**, do belong to one of their arguments.

You already have seen several different methods in the past. Lists, for example, have lots of methods including `append` and `clear`:

In [ ]:
l = []
for i in range(10):
  l.append(i)

print(l)
l.clear()
print(l)

We can tell that `append` and `clear` are methods because they are called differently: rather than using the syntax `append(l, i)`, we use the "dot" syntax `l.append(i)`. This dot syntax is to make it clear that the `append` method belongs to the list `l`. Not only does it not make sense to append to anything that's not a list, but `append` isn't even defined for things that aren't lists. That is, you can't call `append` like this:

In [ ]:
append(l, 1)

Unlike append, there are *some* other types of things that you can `clear`, however. Let's try it with a dictionary:

In [ ]:
d = {'a': 1, 'b': 2, 'c': 3}
print(d)
d.clear()
print(d)

What's going on here? Well, the gist is that both `d` and `l` are *objects* that belong to a *class*. `l` is an object that belongs to the class `list`, and `d` is an object that belongs to the class `dict`. We can confirm this with the function `type`:

In [ ]:
print(type(l))
print(type(d))

To understand what a method is, then, we need to understand what a class is. Basically, a class is a type of object in Python. Objects are instances of a class that have *attributes* and *methods*. *Attributes* are just variables that are a part of an object. You can think of objects like dictionaries and attributes like keys in that dictionary: attributes are just slots to store some data. Finally, as we saw above, *methods* are functions that belong to a class. It's easiest to see how this all works in action, so let's start off with a simple example.

## **BONUS:** Classes - points in space

Let's say that you are presenting stimuli where you need to draw dots at different points on the screen. Points are just (x, y) coordinates, so if we have a bunch of points to draw, we might decide to store these coordinates in two different lists:

In [ ]:
x = [43, 61, 29, 49, 50]
y = [82, 27, 73, 56, 39]

for i in range(len(x)):
  print(f'Point {i}: ({x[i]}, {y[i]})')

For many purposes, this representation is totally fine. But if you're working a lot with these points, you'll want to define functions for them. Here's a function, for example, that translates a point:

In [ ]:
def translate(x, y, xshift=0, yshift=0):
  """Translate a point (x, y) by (xshift, yshift)"""
  return (x+xshift, y+yshift)

x_new = []
y_new = []
for i in range(len(x)):
  new_point = translate(x[i], y[i], xshift=10, yshift=-10)
  x_new.append(new_point[0])
  y_new.append(new_point[1])
  print(f'Point {i}: ({x_new[i]}, {y_new[i]})')

As we can see, this works, but it's clunky. For one, you have to keep two separate lists every time you translate a point. You also have to upack the x and y values returned from `translate` and append them to the new lists individually, which requires you to know that the x value is returned first, and the y value is returned second. One way to improve this is to use a list of lists:

In [ ]:
points = [[43, 82], [61, 27], [29, 73], [49, 56], [50, 39]]

def translate(point, xshift=0, yshift=0):
  """Translate a point [x, y] by (xshift, yshift)"""
  return [point[0]+xshift, point[1]+yshift]

p_new = []
for i in range(len(points)):
  p_new.append(translate(points[i], xshift=10, yshift=-10))
  print(f'Point {i}: ({p_new[i][0]}, {p_new[i][1]})')

This gives us the same result, but it's a little bit easier to work with since we only have one list to manage. But our `translate` function makes a hidden assumption: it assumes that the argument `point` is a list of exactly two numbers. And still, whoever uses the `translate` function needs to know that the points are represented with the x-value first and the y-value second. We can fix these problems by defining a class:

In [ ]:
class Point2D:
  def __init__(self, x=0, y=0):
    self.x = x
    self.y = y

  def translate(self, xshift=0, yshift=0):
    self.x += xshift
    self.y += yshift
    return self

  def __str__(self):
    return '(' + str(self.x) + ', ' + str(self.y) + ')'

There's quite a bit here, so let's break it down. First, the `class` keyword makes a new class, in this case one called `Point2D`. Inside the class definition, we have a bunch of function definitions. These functions are the methods of the class: they belong to the class, and can only be called on objects of that class. The first argument to every method must be `self`, which refers to the object that the method is being called on (i.e., the thing before the dot). 

The first is a method called `__init__`, also known as a constructor. Every Python class needs an `__init__` method, which is responsible for constructing new objects belonging to the class. We can see that `__init__` makes two attributes for the new object, called `x` and `y`. To nobody's surprise, these attributes store the x and y coordinates of the point.

The next method is our `translate` function from before. As before, this just adds `xshift` to the x-value and `yshift` to the y-value. The only difference is that this version uses the `self.x` and `self.y` attributes of the point.

Finally, we defined a method called `__str__`. This isn't strictly necessary, but it allows us to easily print out the points in the format `"(x-value, y-value"`.

Okay, now that we have a class for points, what can we do with it? First, we call the `__init__` constructor using the name of the class as a function (weird, I know):

In [ ]:
p = Point2D(1, 4)
print(p)

We can access the x-value and y-value attributes of the point using the dot syntax:

In [ ]:
print(p.x)
print(p.y)

Finally, we can also call the method `translate` using the dot syntax:

In [ ]:
print(p.translate(10, 10))

Putting this all together, here how we can translate a list of points:

In [ ]:
points = [Point2D(43, 82), Point2D(61, 27), Point2D(29, 73), Point2D(49, 56), Point2D(50, 39)]
for i in range(len(points)):
  print(f'Point {i}: {points[i].translate(10, -10)}')

Just like how functions abstract the details about how the function works, classes and methods are nice because they abstract how the points are represented and how the methods themselves work. When we're translating points, we don't need to think about lists or attributes: we can just know that we're translating a point, and that's enough! We'll talk more about classes and methods later, but by now you've most of what they have to offer

# Putting it all together

Today we covered a **lot** of material about functions. To wrap up, let's discuss some questions as a group:

- What is a function, and what are functions good for?
- Can we get by without writing functions? Should we?
- What's the difference between a function defined using `def` and one using `lambda`?
- How should we test our functions?
- How is a function different from a method?
- When might we prefer to make a function a method?
